In [1]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
from helper import *
metric_to_calculate = ['DTW', 'FID', 'TWED', 'RMSE', 'MMD']

In [2]:
def discriminator(data_dim, input_classes=3):
    
    in_label = Input(shape=(1,))
    x = Embedding(input_classes, 30)(in_label)
    x = Dense(data_dim)(x)
    x = Reshape((data_dim,1))(x)
    
    D_in = Input(shape=[data_dim,1])
    inp1 = Concatenate()([D_in, x])

    x = Conv1D(filters=48, kernel_size=19, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=64, kernel_size=15, padding='same', strides=3, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=80, kernel_size=11, padding='same', strides=2, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=96, kernel_size=9, padding='same', strides=2, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=112, kernel_size=7, padding='same', strides=2, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = GlobalAveragePooling1D()(x)

    x1 = Conv1D(filters=48, kernel_size=9, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=64, kernel_size=7, padding='same', strides=3, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=80, kernel_size=5, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=96, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=112, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = GlobalAveragePooling1D()(x1)

    xx = concatenate([x,x1])

    xx = Dense(100)(xx)
    xx = Dense(100)(xx)

    out = Dense(1, activation='sigmoid')(xx)

    model = Model(inputs=[D_in, in_label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model
    # model.summary()

# d_model = discriminator(data_dim=186, input_classes=3)
# plot_model(d_model, to_file='disc.png', show_shapes=True)

In [3]:
def generator(noise_dim=186, input_classes=3, out_dim=186):
    
    in_label = Input(shape=(1,))
    x = Embedding(input_classes, 30)(in_label)
    x = Dense(noise_dim)(x)
    x = Reshape((noise_dim,1))(x)
    
    G_in = Input(shape=[noise_dim,])
    gen = Dense(noise_dim)(G_in)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((noise_dim,1))(gen)

    x = Concatenate()([gen, x])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=15, strides=1, padding='valid', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    out = Activation('tanh')(x)
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=100, input_classes=3, out_dim=186)
# plot_model(g_model, to_file='gen.png', show_shapes=True)

In [4]:
def create_gan(d_model, g_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_beat = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_beat])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_beat], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt)
    return model

# gan_model = create_gan(d_model, g_model)
# plot_model(gan_model, to_file='gan.png', show_shapes=True)

In [5]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [6]:
def load_real_samples():
    X = np.load('Data/ForGAN/X.npy')
    y = np.load('Data/ForGAN/y.npy')

    # print (X.shape, y.shape)

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    return reshape(X_N), y_N, reshape(X_S), y_S, reshape(X_V), y_V

In [7]:
def generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_samples):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], int(n_samples/3))
    i_S = randint(0, y_S.shape[0], int(n_samples/3))
    i_V = randint(0, y_V.shape[0], int(n_samples/3))
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    labels = np.hstack((y_N[i_N], y_S[i_S], y_V[i_V]))
    # print (labels.shape)
    
    # generate class labels
    y = reshape(np.random.uniform(0.8, 1, n_samples))

    return [X, labels], y

In [8]:
# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    # generate labels
    labels_fake = np.hstack((np.zeros(int(n_samples/3)), np.ones(int(n_samples/3)), 2*np.ones(int(n_samples/3))))
    # np.random.shuffle(labels_fake)
    return [reshape(X_fake), labels_fake]

In [9]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    ecgs = generator.predict([z_input, labels_input])
    # create class labels
    y = reshape(np.random.uniform(0, 0.2, n_samples))
#     y = y.reshape(y.shape[0], 1)
#     y = np.zeros((n_samples, 1))
    return [ecgs, labels_input], y

In [10]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

In [11]:
def save_new_plot(X_R, z_input, n_batch, name):
    n = 3
    Win = (n_batch//3)
    XX = np.vstack((X_R, z_input[0:n,:,:], z_input[Win:Win+n,:,:], z_input[2*Win:2*Win+n,:,:]))
    plt.figure(figsize=(15,5))
    for i in range(n):
        # subplot(R, C, Plot_No)
        plt.subplot(n+1, n, 1 + i)
        plt.axis('off')
        plt.plot(XX[i,:,:])
    for i in range(n, ((n+1)*(n+1))-(n+1)):
        # define subplot
        plt.subplot(n+1, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(XX[i,:,:])
    # plt.show()
    plt.savefig(name, dpi=75)
    plt.close()

In [12]:
# size of the latent space
latent_dim = 100
# size of the data
data = 186
# classes
classes = 3

n_epochs=10

# multiples of three (three classes) (less thyan 24000)
n_batch=300

# Loss Values
G_L = np.infty

# create the discriminator
d_model = discriminator(data_dim=data, input_classes=classes)
# d_model = discriminator(data_dim=data, input_classes=classes)

# create the generator
g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)

# create the gan
gan_model = create_gan(d_model, g_model)

folder_name = 'Final/Beat_Label_Input_New/'
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

plot_model(d_model, to_file=folder_name+'disc.pdf', show_shapes=True)
plot_model(g_model, to_file=folder_name+'gen.pdf', show_shapes=True)
plot_model(gan_model, to_file=folder_name+'gan.pdf', show_shapes=True)


# load image data
X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim)

bat_per_epo = int(y_S.shape[0] / n_batch)
half_batch = int(n_batch / 2)

plt.ioff()


    
filename = folder_name + 'Plots'
if not os.path.isdir(filename):
    os.mkdir(filename)

model_name = folder_name + 'Model/'
if not os.path.isdir(model_name):
    os.mkdir(model_name)

f = open(folder_name + 'Loss.csv', 'w')
f.write('d_loss1, d_loss2, g_loss \n')
f.close()

f = open(folder_name + 'Stats.csv', 'w')
for i in range(3):
    for mtc in metric_to_calculate:
        f.write(str(mtc)+'_'+str(i)+',')
f.write('\n')
f.close()

# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, half_batch)
        # print (X_real.shape, labels_real.shape, y_real.shape)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
        
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # print (X_fake.shape, labels.shape, y_fake.shape)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
        
        # prepare points in latent space as input for the generator
        [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
        [X, _], _ = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_batch)
        # print (z_input.shape)
        # create inverted labels for the fake samples
        y_gan = reshape(np.random.uniform(0.8, 1, n_batch))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
        
#         if g_loss < G_L:
#             G_L = g_loss
#             g_model.save(model_name + str(i*1000 + j) + '_cgan_generator.h5')

        f = open(folder_name + 'Loss.csv', 'a')
        f.write(str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+'\n')
        f.close()

        if (j+1)%2 == 0:
            print('>%d, %d/%d, d1=%.5f, d2=%.5f g=%.5f' %(i, j, bat_per_epo, d_loss1, d_loss2, g_loss))
            name = filename+'/'+str(i*1000 + j)+'.jpg'
            # generate ECGs
            z_input  = g_model.predict([z_input, labels_input])
            X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
            save_new_plot(X_R, z_input, n_batch, name)
            g_model.save(model_name + str(i*1000 + j) + '_cgan_generator.h5')
            
        if (j+1)%10 == 0:
            evaluate(X, z_input, classes, metric_to_calculate, n_batch, folder_name, samples=10)

>0, 1/80, d1=0.97522, d2=0.43001 g=1.63823
>0, 3/80, d1=0.48443, d2=1.58549 g=0.54937
>0, 5/80, d1=0.84997, d2=0.47714 g=1.29009
>0, 7/80, d1=0.90097, d2=0.49759 g=1.13633
>0, 9/80, d1=0.83619, d2=0.52573 g=1.09173
>0, 11/80, d1=0.81130, d2=0.55323 g=0.96344
>0, 13/80, d1=0.70176, d2=0.61745 g=0.81497
>0, 15/80, d1=0.60712, d2=0.73726 g=0.80027
>0, 17/80, d1=0.68371, d2=0.64074 g=0.82758
>0, 19/80, d1=0.65289, d2=0.67242 g=0.75146
>0, 21/80, d1=0.59441, d2=0.68014 g=0.76877
>0, 23/80, d1=0.61299, d2=0.67304 g=0.82227
>0, 25/80, d1=0.58950, d2=0.69590 g=0.77023
>0, 27/80, d1=0.58716, d2=0.69152 g=0.75829
>0, 29/80, d1=0.54684, d2=0.68724 g=0.78277
>0, 31/80, d1=0.54449, d2=0.67366 g=0.82637
>0, 33/80, d1=0.53516, d2=0.69575 g=0.77156
>0, 35/80, d1=0.53041, d2=0.67395 g=0.77708
>0, 37/80, d1=0.53781, d2=0.65614 g=0.83705
>0, 39/80, d1=0.51355, d2=0.68894 g=0.81307
>0, 41/80, d1=0.51715, d2=0.68891 g=0.79079
>0, 43/80, d1=0.51935, d2=0.67032 g=0.83680
>0, 45/80, d1=0.46738, d2=0.69818 g=0

>4, 55/80, d1=0.63977, d2=0.57898 g=1.09245
>4, 57/80, d1=0.58832, d2=0.55895 g=1.13440
>4, 59/80, d1=0.60127, d2=0.54034 g=1.06825
>4, 61/80, d1=0.62110, d2=0.58272 g=1.07938
>4, 63/80, d1=0.54955, d2=0.58568 g=1.10010
>4, 65/80, d1=0.59350, d2=0.61725 g=1.11098
>4, 67/80, d1=0.64303, d2=0.56220 g=1.04707
>4, 69/80, d1=0.66289, d2=0.54832 g=1.14408
>4, 71/80, d1=0.62512, d2=0.62736 g=1.05758
>4, 73/80, d1=0.63646, d2=0.57785 g=1.05733
>4, 75/80, d1=0.65089, d2=0.54741 g=1.06115
>4, 77/80, d1=0.64254, d2=0.74115 g=1.11587
>4, 79/80, d1=0.73462, d2=0.56038 g=1.06260
>5, 1/80, d1=1.01264, d2=0.55629 g=1.14641
>5, 3/80, d1=0.67236, d2=0.62022 g=0.98863
>5, 5/80, d1=0.66290, d2=0.62633 g=0.98415
>5, 7/80, d1=0.74153, d2=0.62376 g=1.04844
>5, 9/80, d1=0.59975, d2=0.55760 g=1.02636
>5, 11/80, d1=0.64611, d2=0.75926 g=1.31862
>5, 13/80, d1=0.82204, d2=0.53660 g=1.19180
>5, 15/80, d1=0.68950, d2=0.66401 g=0.94101
>5, 17/80, d1=0.68964, d2=0.57958 g=0.95345
>5, 19/80, d1=0.62559, d2=0.59665 g=0

>9, 29/80, d1=0.59178, d2=0.52512 g=1.07994
>9, 31/80, d1=0.61380, d2=0.52748 g=1.05297
>9, 33/80, d1=0.60146, d2=0.53723 g=1.07683
>9, 35/80, d1=0.56691, d2=0.52756 g=1.10895
>9, 37/80, d1=0.57610, d2=0.56061 g=1.09297
>9, 39/80, d1=0.67088, d2=0.60951 g=1.08345
>9, 41/80, d1=0.62852, d2=0.59275 g=1.15005
>9, 43/80, d1=0.68393, d2=0.52137 g=1.10786
>9, 45/80, d1=0.64437, d2=0.57651 g=1.19450
>9, 47/80, d1=0.66319, d2=0.58293 g=1.13141
>9, 49/80, d1=0.69238, d2=0.52293 g=1.11529
>9, 51/80, d1=0.65929, d2=0.52621 g=1.18219
>9, 53/80, d1=0.66173, d2=0.59164 g=1.10044
>9, 55/80, d1=0.68160, d2=0.51874 g=1.22773
>9, 57/80, d1=0.62622, d2=0.67587 g=1.22710
>9, 59/80, d1=0.65556, d2=0.60072 g=1.03850
>9, 61/80, d1=0.67006, d2=0.53498 g=1.08084
>9, 63/80, d1=0.59303, d2=0.56081 g=1.00967
>9, 65/80, d1=0.60999, d2=0.53393 g=1.04618
>9, 67/80, d1=0.62005, d2=0.53468 g=1.03580
>9, 69/80, d1=0.61814, d2=0.52268 g=1.04088
>9, 71/80, d1=0.63075, d2=0.55271 g=1.05624
>9, 73/80, d1=0.64199, d2=0.5405